#### Main methodology results

*Disclaimer*: please note that many parts of this code require the preprocessed data from ADNI (both genetic and diagnostic related) as input. This data has not been uploaded to the repository for privacy reasons.

In [4]:

import pandas as pd
import datetime, pickle
import create_datasets.create_nx_datasets
from ml_models.machine_learning_models import create_class_LOAD, baseline_model


**1. Obtain genes of interest**

Using DisGeNET to get Gene-Disease-Associations (GDAs) to Alzheimer's Disease (AD gene set) and other neurodegenerative diseases (ND). This is already obtained from [first part of the results](1_main_methodology.ipynb).

**2. Obtain biological networks**

Using genes of interst obtained from DisGeNET, obtain PPI between them from STRING. This is already obtained from [first part of the results](1_main_methodology.ipynb).

**3. Data preprocessing**

Please refer to `data_preprocessing` subdirectory for this part.
1. [make_BED_files.R](data_preprocessing/make_BED_files.R) creates BED files with the genomic coordinates of the genes of interest. This is already obtained from [first part of the results](1_main_methodology.ipynb).
2. [extract_and_annotate_missense_LOAD.sh](data_preprocessing/extract_and_annotate_missense_LOAD.sh) is the script for obtaining missense variants from the VCF files.

**4. Create graph datasets**

Create graph datasets (one graph representing each patient) for different targets with ADNI dataset.

In [5]:
dataset = 'LOAD'
target  = 'LOAD'
diseases = ['ND']
network = 'original'

for disease in diseases:

    indir = 'data'
    outdir = f'data/graph_datasets/{target}'
    print('Input directory:', indir)
    print('Output directory:', outdir)
    print()

    start_time = datetime.datetime.now()
    print()

    result_nodes = create_datasets.create_nx_datasets.main(indir, dataset, target, disease, network, 'missense', None)
    print('Coding: number of missense variants per node')

    outfile = f'{outdir}/{disease}_PPI_missense.pkl'
    print('Resulting dataset saved at:', outfile)
    print()

    with open(outfile, 'wb') as f:
        pickle.dump(result_nodes, f)

    result_nodes_time = datetime.datetime.now()
    print('Processing time:', result_nodes_time - start_time)
    print('\n\n')

Input directory: data
Output directory: data/graph_datasets/LOAD


data/ND_STRING_PPI_edgelist.txt
Network used: ND original
# nodes = 139
# edges = 263

Dataset used: LOAD
    CHROM        POS          ID REF ALT Allele       Consequence    IMPACT  \
0       1   11854476   rs1801131   A   C      C  missense_variant  MODERATE   
1       1   20977000   rs1043424   A   C      C  missense_variant  MODERATE   
2       1  209782343  rs11119314   T   C      C  missense_variant  MODERATE   
3       2  210558162    rs741006   G   A      A  missense_variant  MODERATE   
4       2  234113301      rs9247   C   T      T  missense_variant  MODERATE   
5       3   12393125   rs1801282   C   G      G  missense_variant  MODERATE   
6       3  133475722   rs1799852   G   A      A  missense_variant  MODERATE   
7       3  133494354   rs1049296   C   T      T  missense_variant  MODERATE   
8       4   23815662   rs8192678   C   T      T  missense_variant  MODERATE   
9       4   23815681  rs17574213   C 

KeyboardInterrupt: 

**5. Graph classification with GNNs**

We then evaluated and tested different GNNs in the framework called [GraphGym](https://github.com/snap-stanford/GraphGym) (You *et al.*, 2020).

Configuration and grid files employed are in the subdirectory [graphgym_files](graphgym_files).

Summarized results obtained by GraphGym in LOAD dataset are [here](results/GNNs_LOAD/2022_02_LOAD.csv)

**6. Baseline model**

Using only APOE gene as input.

In [14]:
target = 'LOAD'

infile = f'data/table_datasets/ND_PPI_missense_with_biomarkers_LOAD.csv'
data = pd.read_csv(infile, index_col = 0)
data_wclass = create_class_LOAD(data)

x = data_wclass.drop(columns=['y'])
x = x['APOE']

y = data_wclass['y']
x.index = x.index.str.upper()

f = open(f'data/splits/split_{target}.pkl', 'rb')
split_load = pickle.load(f)
f.close()

auc_load = baseline_model(split_load, x, y)
print('Baseline model LOAD, AUC ROC:', auc_load)

Class distribution:
1.0    1014
0.0     585
Name: y, dtype: int64

Confusion matrix:
 [[  0  59]
 [  0 101]]

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        59
         1.0       0.63      1.00      0.77       101

    accuracy                           0.63       160
   macro avg       0.32      0.50      0.39       160
weighted avg       0.40      0.63      0.49       160

Baseline model LOAD, AUC ROC: 0.5402752139620742


/home/laura/anaconda3/envs/graphgym/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/laura/anaconda3/envs/graphgym/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/laura/anaconda3/envs/graphgym/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r